In [1]:
import csv
import re
import time
import datetime
import os

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import pandas as pd
from openpyxl import load_workbook

import configparser 
config = configparser.ConfigParser()
config.read('config.ini')
ip = config['DEFAULT']['IP']
port = config['DEFAULT']['MongoDB-Port']

from pymongo import MongoClient
client = MongoClient(ip, int(port))

In [2]:
db_twitter = client["Twitter"]
collections_twitter = db_twitter.collection_names()

In [3]:
current_timestamp = int(time.time() * 1000)
current_year = int(datetime.datetime.now().year)
print("current year : " + str(current_year))

current_week = int((current_timestamp - 1546214400000)/1000/604800)+1
print("current week : " + str(current_week))

current year : 2019
current week : 11


In [4]:
dic_collection = {}
for i in collections_twitter:
    if i.startswith("20"):
        year = i[0:4]
        week = re.search('_(.+?)_', i).group(1)[1:]
        if int(year) < current_year:
            dic_collection[i] = "{:,}".format(db_twitter[i].find({}).count())
        else:
            try:
                if int(week) < current_week:
                    dic_collection[i] = "{:,}".format(db_twitter[i].find({}).count())
            except: pass

for key in sorted(dic_collection):
    print("%s: %s" % (key, dic_collection[key]))

2018_W51_Twitter_Australia: 20,471
2018_W51_Twitter_Other: 8,978,243
2018_W52_Twitter_Australia: 38,065
2018_W52_Twitter_Other: 16,940,708
2019_W10_Twitter_Australia: 40,767
2019_W10_Twitter_Other: 15,941,350
2019_W1_Twitter_Australia: 40,880
2019_W1_Twitter_Other: 16,994,959
2019_W2_Twitter_Australia: 37,645
2019_W2_Twitter_Other: 14,474,627
2019_W3_Twitter_Australia: 52,348
2019_W3_Twitter_Other: 20,416,992
2019_W4_Twitter_Australia: 59,625
2019_W4_Twitter_Other: 23,417,927
2019_W5_Twitter_Australia: 61,617
2019_W5_Twitter_Other: 23,917,345
2019_W6_Twitter_Australia: 61,435
2019_W6_Twitter_Other: 24,176,122
2019_W7_Twitter_Australia: 28,017
2019_W7_Twitter_Other: 10,484,648
2019_W8_Twitter_Australia: 60,035
2019_W8_Twitter_Other: 23,446,630
2019_W9_Twitter_Australia: 58,316
2019_W9_Twitter_Other: 22,293,795


# Extract a hashtag histogram from individual collections

## 1: iterate all records which have hashtag

In [5]:
pipeline = [
    {"$match": { "entities.hashtags": {"$exists":True,"$ne":[]}}},
    {"$match": { "lang" : "en"}},
    { "$group": {
        "_id": "$entities.hashtags",
        "count": { "$sum": 1 },
        }
    }
]

In [10]:
csv_columns = ['hashtag','count']

for collection in sorted(dic_collection):
    print("-------------------\n")
    print("Processing on collection: " + collection)
    
    hashtag_list={}
    data = list(db_twitter[collection].aggregate(pipeline,allowDiskUse=True))
    if len(data) > 0 : 
        for i in range(len(data)):
            for j in data[i]["_id"]:
                h = j["text"].lower()
                if(re.match("^[a-zA-Z0-9]*$",h)):
                    if len(hashtag_list)>0:
                        if h in hashtag_list:
                            hashtag_list[h] += data[i]["count"]
                        else:
                            hashtag_list[h] = data[i]["count"]
                    else:
                        hashtag_list[h] = data[i]["count"]
    print("hashtag_list is finished")
    
    #create folder if not exist
    folder = "output/hashtag_histogram/"
    if not os.path.exists(folder):
        os.makedirs(folder)
        
    csv_file = "output/hashtag_histogram/{" + collection + "}_hashtag_histogram.csv"
    try:
        with open(csv_file, 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
            writer.writeheader()
            for key in hashtag_list.keys():
                csvfile.write("%s,%s\n"%(key,hashtag_list[key]))
        print("csvfile for collection "+ collection + "is done.")
        print("-------------------\n")
    except IOError:
        print("I/O error")
        print("-------------------\n")
        

-------------------

Processing on collection: 2018_W51_Twitter_Other
hashtag_list is finished
csvfile for collection 2018_W51_Twitter_Otheris done.
-------------------

-------------------

Processing on collection: 2018_W52_Twitter_Other
hashtag_list is finished
csvfile for collection 2018_W52_Twitter_Otheris done.
-------------------

-------------------

Processing on collection: 2019_W10_Twitter_Other
hashtag_list is finished
csvfile for collection 2019_W10_Twitter_Otheris done.
-------------------

-------------------

Processing on collection: 2019_W1_Twitter_Other


KeyboardInterrupt: 